In [13]:
import pandas as pd
from pathlib import Path
import os

In [14]:
DATA_DIR = Path(os.environ.get("DATA_DIR"))
LANDING_DIR = DATA_DIR / "landing"
RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(exist_ok=True, parents=True)
xlsx_files = list(LANDING_DIR.glob("For RO.xlsx"))
landing_file = xlsx_files[0]

In [15]:
df_master = pd.read_excel(
    landing_file,
    sheet_name="Sheet1",
    header=1,
).iloc[:24]
df_master = df_master.rename(columns={"Meta ID#": "mouse_name"})
df_master.head()

,mouse_name,Fear ID#,Video ID#,CondBase,CondT1,CondT3,ExtBase,ExtT1,ExtT2,ExtT3,...,groom.2,hang.2,sniff.2,rear.2,rest.2,walk.2,eatpaw.1,Explore.2,Cons.2,Low.2
0,F1,A-1,cam_17202339,0.000000,0.000000,83.333333,0.0,73.333333,43.333333,16.666667,...,0.098719,0.040243,0.216039,0.005556,0.459120,0.016362,0.042859,0.062161,0.262680,0.675159
1,F2,A-2,cam_17202342,0.000000,16.666667,100.000000,0.0,76.666667,33.333333,26.666667,...,0.051222,0.022651,0.248303,0.005942,0.511283,0.010391,0.035568,0.038984,0.201430,0.759585
2,F3,A-3,cam_17202338,5.555556,16.666667,50.000000,0.0,30.000000,26.666667,10.000000,...,0.088434,0.027735,0.223422,0.006605,0.526203,0.017161,0.018933,0.051500,0.198875,0.749625
3,F4,A-4,cam_17202346,2.777778,50.000000,83.333333,0.0,66.666667,50.000000,36.666667,...,0.166175,0.036379,0.218759,0.009371,0.432190,0.020518,0.023311,0.066268,0.282784,0.650948
4,F5,A-5,cam_17202341,2.777778,33.333333,83.333333,0.0,53.333333,26.666667,13.333333,...,0.139017,0.029312,0.236538,0.008009,0.417968,0.015496,0.027731,0.052817,0.292677,0.654506


In [16]:
mouse_cols = ["mouse_name", "Fear ID#", "Video ID#"]

df_mice = df_master.loc[:, mouse_cols]
df_mice = df_mice.rename(columns={"Fear ID#": "fear_id", "Video ID#": "video_id"})
df_mice.head(3)

df_mice.to_csv(RAW_DIR / "fc_mice.csv", index=False)

In [17]:
fear_cond_cols = ["mouse_name", "CondBase", "CondT1", "CondT3"]

df_fear_cond = df_master.loc[:, fear_cond_cols]
df_fear_cond = df_fear_cond.rename(
    columns={"CondBase": "cond_base", "CondT1": "cond_t1", "CondT3": "cond_t3"}
)
df_fear_cond.head(3)

df_fear_cond.to_csv(RAW_DIR / "fc_freeze_fear_conditioning.csv", index=False)

In [18]:
extinction_cols = ["mouse_name", "ExtBase"] + [f"ExtT{i}" for i in range(1, 11)]

df_extinction = df_master.loc[:, extinction_cols]
mapper = {"ExtBase": "ext_base"}
mapper.update({f"ExtT{i}": f"ext_t{i}" for i in range(1, 11)})
df_extinction = df_extinction.rename(columns=mapper)
df_extinction.head(3)

df_extinction.to_csv(RAW_DIR / "fc_freeze_extinction.csv", index=False)

In [19]:
ext_ret_cols = ["mouse_name", "ExtRetBase", "ExtRetAv"]

df_ext_ret = df_master.loc[:, ext_ret_cols]
df_ext_ret = df_ext_ret.rename(
    columns={"ExtRetBase": "ext_ret_base", "ExtRetAv": "ext_ret_av"}
)
df_ext_ret.head(3)

df_ext_ret.to_csv(RAW_DIR / "fc_freeze_ext_ret.csv", index=False)

In [20]:
renewal_cols = ["mouse_name", "RenBase", "RenAv"]

df_renewal = df_master.loc[:, renewal_cols]
df_renewal = df_renewal.rename(columns={"RenBase": "ren_base", "RenAv": "ren_av"})
df_renewal.head(3)

df_renewal.to_csv(RAW_DIR / "fc_freeze_renewal.csv", index=False)

In [21]:
df_behavior = pd.concat(
    [
        df_fear_cond,
        df_extinction.drop("mouse_name", axis=1),
        df_ext_ret.drop("mouse_name", axis=1),
        df_renewal.drop("mouse_name", axis=1),
    ],
    axis=1,
)
df_behavior.head(3)

df_behavior.to_csv(RAW_DIR / "fc_freeze_behavior.csv", index=False)

### Home cage

In [22]:
MOUSE_NAME_DF = df_mice[["mouse_name"]]


df_pre_cond = pd.read_excel(
    landing_file, sheet_name="Sheet1", header=1, usecols="V:AG"
).iloc[:24]

df_pre_cond = pd.concat([MOUSE_NAME_DF, df_pre_cond], axis=1)
df_pre_cond = df_pre_cond.rename(
    columns={"Explore": "agg_explore", "Cons": "agg_cons", "Low": "agg_low"}
)

df_pre_cond.to_csv(RAW_DIR / "fc_hc_pre_conditioning.csv", index=False)
df_pre_cond.head()

,mouse_name,drink,eat,groom,hang,sniff,rear,rest,walk,eatpaw,agg_explore,agg_cons,agg_low
0,F1,0.000905,0.074037,0.055741,0.053475,0.217713,0.006973,0.552253,0.019494,0.019410,0.079941,0.150092,0.769966
1,F2,0.001171,0.099765,0.065576,0.047145,0.224245,0.008726,0.519748,0.014983,0.018642,0.070854,0.185154,0.743992
2,F3,0.000623,0.161955,0.082278,0.060919,0.161275,0.007427,0.496778,0.015108,0.013638,0.083454,0.258493,0.658053
3,F4,0.001212,0.215785,0.077888,0.040694,0.211751,0.010348,0.406498,0.020437,0.015386,0.071479,0.310272,0.618250
4,F5,0.001845,0.095068,0.055748,0.067578,0.294497,0.011249,0.442341,0.017227,0.014447,0.096054,0.167109,0.736838


In [23]:
df_post_cond = pd.read_excel(
    landing_file, sheet_name="Sheet1", header=1, usecols="AH:AS"
).iloc[:24]

df_post_cond = pd.concat([MOUSE_NAME_DF, df_post_cond], axis=1)

column_mapper = {
    col: f"{col.split('.')[0]}" for col in df_post_cond.columns if "." in col
}
df_post_cond = df_post_cond.rename(columns=column_mapper)
df_post_cond = df_post_cond.rename(
    columns={"Explore": "agg_explore", "Cons": "agg_cons", "Low": "agg_low"}
)

df_post_cond.to_csv(RAW_DIR / "fc_hc_post_conditioning.csv", index=False)
df_post_cond.head()

,mouse_name,drink,eat,groom,hang,sniff,rear,rest,walk,eathand,agg_explore,agg_cons,agg_low
0,F1,0.001922,0.091809,0.101830,0.029973,0.293462,0.007910,0.423609,0.023301,0.026184,0.061183,0.221745,0.717071
1,F2,0.001552,0.092590,0.058673,0.029048,0.234879,0.008876,0.523999,0.015435,0.034949,0.053359,0.187763,0.758878
2,F3,0.000491,0.094036,0.073905,0.023333,0.234628,0.007746,0.527250,0.017590,0.021021,0.048670,0.189452,0.761878
3,F4,0.001722,0.101337,0.088967,0.025945,0.278413,0.009604,0.441626,0.024902,0.027484,0.060450,0.219510,0.720040
4,F5,0.001708,0.098956,0.077420,0.039060,0.246052,0.008916,0.486566,0.020194,0.021129,0.068171,0.199212,0.732618


In [24]:
df_post_ext = pd.read_excel(
    landing_file, sheet_name="Sheet1", header=1, usecols="AT:BE"
).iloc[:24]

df_post_ext = pd.concat([MOUSE_NAME_DF, df_post_ext], axis=1)

column_mapper = {
    col: f"{col.split('.')[0]}" for col in df_post_ext.columns if "." in col
}
df_post_ext = df_post_ext.rename(columns=column_mapper)
df_post_ext = df_post_ext.rename(
    columns={"Explore": "agg_explore", "Cons": "agg_cons", "Low": "agg_low"}
)

df_post_ext.to_csv(RAW_DIR / "fc_hc_post_extinction.csv", index=False)
df_post_ext.head()

,mouse_name,drink,eat,groom,hang,sniff,rear,rest,walk,eatpaw,agg_explore,agg_cons,agg_low
0,F1,0.002571,0.118531,0.098719,0.040243,0.216039,0.005556,0.459120,0.016362,0.042859,0.062161,0.262680,0.675159
1,F2,0.000737,0.113903,0.051222,0.022651,0.248303,0.005942,0.511283,0.010391,0.035568,0.038984,0.201430,0.759585
2,F3,0.000557,0.090950,0.088434,0.027735,0.223422,0.006605,0.526203,0.017161,0.018933,0.051500,0.198875,0.749625
3,F4,0.001518,0.091780,0.166175,0.036379,0.218759,0.009371,0.432190,0.020518,0.023311,0.066268,0.282784,0.650948
4,F5,0.001668,0.124260,0.139017,0.029312,0.236538,0.008009,0.417968,0.015496,0.027731,0.052817,0.292677,0.654506
